In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# load MATLAB files
from scipy.io import loadmat
from scipy.optimize import minimize
from scipy.optimize import fmin_ncg

In [2]:
data = loadmat('data/number.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [3]:
y = data['y']
# Add constant for intercept
X = np.c_[np.ones((data['X'].shape[0],1)), data['X']]
regularization = 0

print('X: {} (with intercept)'.format(X.shape))
print('y: {}'.format(y.shape))

X: (5000, 401) (with intercept)
y: (5000, 1)


In [4]:
sample = np.random.choice(X.shape[0], 20)
plt.imshow(X[sample,1:].reshape(-1,20).T)
plt.axis('off');

In [5]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [6]:
def cost_function(theta,x,y,reg=regularization):
    theta = theta.reshape(-1,1)
    m = y.size
    h = sigmoid((x.dot(theta)).reshape(-1,1))
    j = -(1/m) * (np.log(h).T.dot(y) + np.log(1-h).T.dot((1-y))) + (reg/(2*m))*np.sum(np.square(theta[1:]))
    return j    

In [7]:
def gradient(theta,x,y,reg=regularization):
    theta = theta.reshape(-1,1)
    m = y.size
    h = sigmoid((x.dot(theta)).reshape(-1,1))
    grad = (1/m) * (X.T.dot(h-y)) + (reg/m)*np.r_[[[0]],theta[1:]]
    return grad.flatten()

#### One-vs-all Classification

In [8]:
def oneVsAll(features, classes, n_labels, reg):
    initial_theta = np.zeros((X.shape[1],1))  # 401x1
    all_theta = np.zeros((n_labels, X.shape[1])) #10x401

    for c in np.arange(1, n_labels+1):
        res = minimize(cost_function, initial_theta, args=(features, (classes == c)*1, regularization), method=None,
                       jac=gradient, options={'maxiter':100})
        #optimized = fmin_ncg(f=cost_function, x0=initial_theta, fprime=gradient, args=(features, (classes == c)*1, regularization), maxiter=100)
        all_theta[c-1] = res.x
    return(all_theta)

In [9]:
theta = oneVsAll(X, y, 10, 0.1)

#### One-vs-all Prediction

In [10]:
def predictOneVsAll(all_theta, features):
    probs = sigmoid(X.dot(all_theta.T))
        
    # Adding one because Python uses zero based indexing for the 10 columns (0-9),
    # while the 10 classes are numbered from 1 to 10.
    return(np.argmax(probs, axis=1)+1)

In [11]:
pred = predictOneVsAll(theta, X)
print('Training set accuracy: {} %'.format(np.mean(pred == y.ravel())*100))

probs = sigmoid(X[150].dot(theta.T))
prediction = (np.argmax(probs)+1)
print('150th is {}'.format(prediction))

Training set accuracy: 94.98 %
150th is 10
